In [ ]:
# Connect GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Loading Data from Google Drive

In [ ]:
# Uploading images zip from google drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import io
import zipfile

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download file and unzip
fid = drive.ListFile({'q':"title='br-coins.zip'"}).GetList()[0]['id']
file = drive.CreateFile({'id': fid})

file.GetContentFile('br-coins.zip')
!unzip br-coins.zip -d ./

## Initialization

In [ ]:
import os
import cv2
import numpy as np
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation, BatchNormalization, GlobalAveragePooling2D, TimeDistributed, LSTM
from keras.optimizers import RMSprop, Adagrad, Adam, Adadelta
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.preprocessing import image as preimage  
from keras.wrappers import scikit_learn
from keras.utils import np_utils
from keras.models import load_model

from sklearn import metrics
from sklearn.cross_validation import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from PIL import Image
from math import sqrt

In [ ]:
# Hight and width of the images
IMAGE_SIZE = 100

# 3 channels, Red, Green and Blue
CHANNELS = 3

# Number of epochs
EPOCH = 100

# learning rate
LEARN_RATE = 1.0e-3

# Batch size
BATCH_SIZE = 16

## Functions

In [ ]:
# Read total image from folder
def read_image():
    images = []
    labels = []
    folder_path = 'br-coins/all/'
    
    for image_name in os.listdir(folder_path):
        image_path = folder_path + image_name
        image_label = int(image_name.split('_')[0])
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        images.append(image)
        labels.append(image_label)
            
    return images, labels


# Extract coins from image
def extract_coins(img):
    # Convert to b&w
    cimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Find circles on the image
    circles = cv2.HoughCircles(
        cimg, cv2.HOUGH_GRADIENT, 2, 60, param1=300, param2=30, minRadius=30, maxRadius=50)
    
    # Convert to HSV colorspace
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Define color range for masking
    lower = np.array([0, 0, 0])
    upper = np.array([255, 255, 90])
    # Apply the mask
    mask = cv2.blur(cv2.inRange(hsv, lower, upper), (10, 10))
    
    frames = []
    radiuses = []
    # If circles were not found
    if circles is None:
        return None, None
    
    for circle in circles[0]:
        center_x = int(circle[0])
        center_y = int(circle[1])
        
        # If center of coin lays in masked coin range
        if not mask[center_y, center_x]:
            continue
        
        # Increase radius by C, circle detector tends to decrease radius
        radius = circle[2] + 3
        
        radiuses.append(radius)
        
        # Coordinates of upper left corner of square
        x = int(center_x - radius)
        y = int(center_y - radius)
        
        # As radius was increased the coordinates, could go out of bounds
        if y < 0:
            y = 0
        if x < 0:
            x = 0
                
        frames.append(img[y: int(y + 2 * radius), x: int(x + 2 * radius)])

    return np.array(frames), radiuses


# Rejoint images to a new one
def image_joint(image_list):
    image_size = 376
    size = 0
    img_dict = {}
    count = 0
    
    # Get the size of the biggest one, resize all images with it
    for img in image_list:
        img_dict[count] = img.shape[0] * img.shape[1]
        count += 1
        if img.shape[0] >= size:
            size = img.shape[0]
        if img.shape[1] >= size:
            size = img.shape[1] 
    
    img_dict = sorted(img_dict.items(), key=lambda item:item[1])
    new_image_list = []
    for key, item in img_dict:
        img = image_list[key]
        if img.shape[0] < size or img.shape[1] < size:
            new_img=Image.new('RGB', (size, size), '#FAFAFA')
            new_img.paste(Image.fromarray(img), box=(0, 0, img.shape[1], img.shape[0]))
            new_image_list.append(np.array(new_img))
        else:
            new_image_list.append(img)
            
    # Joint all single images to new one with original size
    new_img=Image.new('RGB', (image_size, image_size), '#FAFAFA')
    
    for i in range(len(new_image_list)):
        img = new_image_list[i]
        
        if i == 0:
            lt = rt = 0
            lb = size
            rb = size
        else:
            if lb + size <= image_size:
                lt += size
                lb += size 
            else:
                lt = 0
                rt += size
                lb = size
                rb += size
                
        new_img.paste(Image.fromarray(img), box=(lt, rt, lb, rb))
            
    return np.array(new_img)


# Plot history for accuracy and loss
def plot_model(model_details):
    # Create sub-plots
    fig, axs = plt.subplots(1, 3, figsize=(20, 5))
    
    # Summarize history for loss
    axs[0].plot(range(1, len(model_details.history['loss'])+1), model_details.history['loss'])
    axs[0].plot(range(1, len(model_details.history['val_loss'])+1), model_details.history['val_loss'])
    axs[0].set_title('Model Loss')
    axs[0].set_ylabel('Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_details.history['loss'])+1), len(model_details.history['loss'])/10)
    axs[0].legend(['train', 'val'], loc='best')

    # Summarize history for mean squared error
    axs[1].plot(range(1, len(model_details.history['mean_squared_error'])+1), model_details.history['mean_squared_error'])
    axs[1].plot(range(1, len(model_details.history['val_mean_squared_error'])+1), model_details.history['val_mean_squared_error'])
    axs[1].set_title('Model Mean Squared Error')
    axs[1].set_ylabel('Mean Squared Error')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_details.history['mean_squared_error'])+1), len(model_details.history['mean_squared_error'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    
    # Summarize history for mean absolute error
    axs[2].plot(range(1, len(model_details.history['mean_absolute_error'])+1), model_details.history['mean_absolute_error'])
    axs[2].plot(range(1, len(model_details.history['val_mean_absolute_error'])+1), model_details.history['val_mean_absolute_error'])
    axs[2].set_title('Model Mean Absolute Error')
    axs[2].set_ylabel('Mean Absolute Error')
    axs[2].set_xlabel('Epoch')
    axs[2].set_xticks(np.arange(1, len(model_details.history['mean_absolute_error'])+1), len(model_details.history['mean_absolute_error'])/10)
    axs[2].legend(['train', 'val'], loc='best')
    
    # Show the plot
    plt.show()

In [ ]:
X_train = np.load('coins_X_train.npy')
Y_train = np.load('coins_Y_train.npy')
X_test = np.load('coins_X_test.npy')
Y_test = np.load('coins_Y_test.npy')
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

## Data Preparation

In [ ]:
# Read all images fron folder
all_image, all_label = read_image()
all_image = np.array(all_image)
all_label = np.array(all_label)

In [ ]:
y = np.bincount(all_label)
ii = np.nonzero(y)[0]
plt.bar(np.unique(all_label), y[ii], width=4, color="#87CEFA")
plt.xlabel('Money (cents)')
plt.ylabel('Amount')
plt.title('Sum amount for each value in money')

In [ ]:
images = []
labels = []
for i in range(len(all_image)):
    image = all_image[i]
    label = all_label[i]

    # Extract coins from image and rejoint to a new image
    prepared, _ = extract_coins(image)

    if prepared is not None and len(prepared):
        # merge extracted coins back to on image
        image = image_joint(prepared)

        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        images.append(image)
        labels.append(label)

all_image_new = np.array(images)
all_label_new = np.array(labels)

In [ ]:
np.save('coins_all_image_new1.npy', all_image_new)
np.save('coins_all_label_new1.npy', all_label_new)
all_image_new.shape, all_label_new.shape

In [ ]:
all_image_new = np.load('coins_all_image_new.npy')
all_label_new = np.load('coins_all_label_new.npy')
all_image_new.shape, all_label_new.shape

In [ ]:
# Splitting
# X_train, X_test, Y_train, Y_test = train_test_split(all_image_new, all_label_new, test_size=0.2, random_state=None)
X_train = np.array([])
X_test = np.array([])
Y_train = np.array([])
Y_test = np.array([])
for label in np.unique(all_label_new):        
    index = np.argwhere(all_label_new == label)
    x_train, x_test, y_train, y_test = train_test_split(all_image_new[index[:,0]], all_label_new[index[:,0]], test_size=0.2, random_state=None)
    if len(X_train) == 0:
        X_train = x_train
        X_test = x_test
        Y_train = y_train
        Y_test = y_test
    else:
        X_train = np.concatenate((X_train, x_train), axis=0)
        X_test = np.concatenate((X_test, x_test), axis=0)
        Y_train = np.concatenate((Y_train, y_train), axis=0)
        Y_test = np.concatenate((Y_test, y_test), axis=0)

# Show first 10 images
print(Y_train[0:10])

slice = 10
plt.figure(figsize=(16, 16))
for i in range(slice):
    plt.subplot(1, slice, i+1)
    plt.imshow(X_train[i], interpolation='nearest')
    plt.axis('off')    
    
print(Y_test[0:10])

slice = 10
plt.figure(figsize=(16, 16))
for i in range(slice):
    plt.subplot(1, slice, i+1)
    plt.imshow(X_test[i], interpolation='nearest')
    plt.axis('off')    
    
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
np.save('coins_X_train.npy', X_train)
np.save('coins_Y_train.npy', Y_train)
np.save('coins_X_test.npy', X_test)
np.save('coins_Y_test.npy', Y_test)

In [ ]:
X_train = np.load('coins_X_train.npy')
Y_train = np.load('coins_Y_train.npy')
X_test = np.load('coins_X_test.npy')
Y_test = np.load('coins_Y_test.npy')
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
y = np.bincount(Y_train)
ii = np.nonzero(y)[0]
plt.bar(np.unique(Y_train), y[ii], width=4, color="#87CEFA")
plt.xlabel('Money (cents)')
plt.ylabel('Count')
plt.title('Training Dataset')

In [ ]:
y = np.bincount(Y_test)
ii = np.nonzero(y)[0]
plt.bar(np.unique(Y_test), y[ii], width=4, color="#87CEFA")
plt.xlabel('Money (cents)')
plt.ylabel('Count')
plt.title('Testing Dataset')

In [ ]:
y_train = np.bincount(Y_train)
ii_train = np.nonzero(y_train)[0]
y_test = np.bincount(Y_test)
ii_test = np.nonzero(y_test)[0]

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.pie(y_train[ii_train], labels=np.unique(Y_train))  
plt.title("Training Dataset") 

plt.subplot(1, 2, 2)
plt.pie(y_test[ii_test], labels=np.unique(Y_test))  
plt.title("Testing Dataset") 

## Modeling

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1))

model.summary()

### 1.Training Layers

In [ ]:
#1
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
#1
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
#1
plot_model(model_details)

In [ ]:
#1
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
#2
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
#2
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
#2
plot_model(model_details)

In [ ]:
#2
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
#3
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
#3
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
#3
plot_model(model_details)

In [ ]:
#3
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
#4
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
#4
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=32),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=32))

In [ ]:
#4
plot_model(model_details)

In [ ]:
#4
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

### 2.Training Batch Size

In [ ]:
# Batch size
BATCH_SIZE = 8

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Batch size
BATCH_SIZE = 16

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Batch size
BATCH_SIZE = 32

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Batch size
BATCH_SIZE = 64

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

### 3.Optimizer

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# RMSprop
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=RMSprop(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Adagrad
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adagrad(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Adadelta
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adadelta(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Adam
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

### 4.Learning Rate

In [ ]:
# learning rate
LEARN_RATE = 1.0e-1

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# learning rate
LEARN_RATE = 1.0e-2

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# learning rate
LEARN_RATE = 1.0e-3

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# learning rate
LEARN_RATE = 1.0e-4

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

### 5.Dropout

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.01))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.01))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

### 6.Final Model

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))   
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1024, activation='linear'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1))

model.summary()

In [ ]:
# Save the model after every epoch
checkpoint = ModelCheckpoint('coins_reg_cnn.final.{epoch:02d}-{loss:.5f}.h5',  # model filename
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True, 
                             mode='auto')

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.1, 
                             patience=10, 
                             verbose=0, 
                             mode='auto', 
                             min_delta=1.0e-5, 
                             cooldown=0, 
                             min_lr=0)
    
model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])

train_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

train_datagen.fit(X_train)

test_datagen = preimage.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')

test_datagen.fit(X_train)

model_details = model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                                    epochs=EPOCH,
                                    verbose=1,
                                    callbacks=[checkpoint, reducelr],
                                    validation_data=test_datagen.flow(X_test, Y_test, batch_size=BATCH_SIZE))

In [ ]:
plot_model(model_details)

In [ ]:
mean = train_datagen.mean
std = train_datagen.std
X_test = X_test.astype(np.float32, copy=False)
X_test -= mean
X_test /= std

score = model.evaluate(X_test, Y_test, verbose=0)

for i in range(0, len(score)):
    print('{}: {}'.format(model.metrics_names[i], score[i]))

In [ ]:
# Predict image with one coin
Y_pred = model.predict(X_test)

error = np.mean(abs(np.array(Y_test) - np.array(Y_pred)))
print("Mean prediction error for images: {0:.2f} cents".format(error))

### 7.Cross Validation

In [ ]:
def create_network():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))   
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(1024, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(1))
    
    model.compile(loss='mse',
              optimizer=Adam(lr=LEARN_RATE),
              metrics=['mse', 'mae'])
    
    return model

In [ ]:
# Normalization
X_train = X_train.astype(np.float32, copy=False)
X_test = X_test.astype(np.float32, copy=False)

mean = X_train.mean(axis=(0, 1, 2))
std = X_train.std(axis=(0, 1, 2))

X_train -= mean
X_train /= std
X_test -= mean
X_test /= std

In [ ]:
#3
seed = 1
np.random.seed(seed)

# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=create_network, 
                           epochs=EPOCH,
                           batch_size=BATCH_SIZE,
                           verbose=1)

kfold = KFold(n_splits=3, random_state=seed)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)

In [ ]:
estimator.fit(X_train, Y_train)

In [ ]:
# Predict image
Y_pred = estimator.predict(X_test)

error = np.mean(abs(np.array(Y_test) - np.array(Y_pred)))
print("Mean prediction error for 3-fold: {0:.2f} cents".format(error))

In [ ]:
#5
seed = 1
np.random.seed(seed)

# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=create_network, 
                           epochs=EPOCH,
                           batch_size=BATCH_SIZE,
                           verbose=1)

kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)

In [ ]:
estimator.fit(X_train, Y_train)

In [ ]:
# Predict image
Y_pred = estimator.predict(X_test)

error = np.mean(abs(np.array(Y_test) - np.array(Y_pred)))
print("Mean prediction error for 5-fold: {0:.2f} cents".format(error))